### Plots for the paper

In [1]:
################## IMPORT LIBRARIES ##################
import sys
import importlib
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import glob
import os
import soundfile as sf
from IPython.display import Audio, display, HTML
import torch
from os.path import join as pjoin


In [8]:
################## IMPORT MY MODULES ##################
sys.path.append('../src')

import helpers as hlp
import evaluation
import dataset as ds
import trainer
import models
import loss_mel, loss_stft, loss_waveform, loss_embedd

importlib.reload(evaluation)
importlib.reload(hlp)
importlib.reload(ds)
importlib.reload(trainer)
importlib.reload(models)
importlib.reload(loss_mel)
importlib.reload(loss_stft)
importlib.reload(loss_waveform)
importlib.reload(loss_embedd)


<module 'loss_embedd' from '/home/ubuntu/joanna/reverb-match-cond-u-net/notebooks/../src/loss_embedd.py'>

In [20]:
################## LOAD CSV WITH EVALUATION RESULTS ##################
df=pd.read_csv("/home/ubuntu/Data/RESULTS-reverb-match-cond-u-net/runs-exp-20-05-2024/210325_evaluation_short.csv")
display(df[df["idx"]==2])

,label,idx,compared,1L_multi-stft-mag,1L_stft-mag,1L_multi-wave,1L_wave,1L_logmel,1L_multi-mel,1S_sisdr,...,2S_pesq,2S_stoi,3D_pesq,3D_stoi,3D_sisdr,3D_mos_nidiff,3D_pesq_nidiff,3D_stoi_nidiff,3D_sisdr_nidiff,3D_srmr_nidiff
4,anecho+fins,2,prediction:target,0.603527,0.598110,0.053656,0.077396,1.063896,0.159209,-0.237343,...,2.648858,0.907209,0.141263,0.018162,5.762266,0.002400,0.001132,0.196261,3.545799,0.237573
5,anecho+fins,2,prediction:content,0.667274,0.667963,0.058213,0.075822,3.590947,0.280614,-1.195281,...,1.987588,0.885286,0.357812,0.015116,5.423813,0.043836,0.050624,1.201276,11.702838,0.587801
20004,dfnet+fins,2,prediction:target,0.633232,0.626679,0.063710,0.088132,1.845446,0.214384,-1.300625,...,2.368681,0.883031,0.335939,0.041331,6.557494,0.009025,0.023871,0.596045,2.924690,0.756307
20005,dfnet+fins,2,prediction:content,0.615798,0.618621,0.056816,0.080161,3.250289,0.279646,0.303117,...,2.326263,0.922564,0.163137,0.038285,6.219041,0.032411,0.025621,0.801492,11.081729,0.406078
40004,wpe+fins,2,prediction:target,0.610761,0.609029,0.077052,0.093164,4.555043,0.324823,-2.089341,...,1.837167,0.838153,0.674187,0.072129,7.160209,1.803069,0.143954,2.194854,12.613714,3.231612
40005,wpe+fins,2,prediction:content,0.570458,0.573144,0.055481,0.064456,1.366249,0.180565,-1.304962,...,2.538981,0.872606,0.175111,0.069083,6.821756,1.761632,0.094462,0.797317,4.456676,2.881383


In [21]:
################## ADD COLUMN TO KNOW IF THE SAMPLE IS REV2DRY OR DRY2REV ##################

# divide into re-reverbaration, de-reverberation 
config=hlp.load_config(pjoin("/home/ubuntu/joanna/reverb-match-cond-u-net/config/basic.yaml"))

def get_reverb_ind(config, df, split):
    config["p_noise"]=0
    config["split"]=split
    dataset=ds.DatasetReverbTransfer(config)
    indices_dry2rev=dataset.get_idx_with_rt60diff(-3,-0.3)
    indices_rev2dry=dataset.get_idx_with_rt60diff(0.3,3)
    indices_smalldiff=dataset.get_idx_with_rt60diff(-0.3,0.3)
    df.loc[df["idx"].isin(indices_dry2rev), "rev_delta"] = "dry2rev"
    df.loc[df["idx"].isin(indices_rev2dry), "rev_delta"] = "rev2dry"
    df.loc[df["idx"].isin(indices_smalldiff), "rev_delta"] = "smalldiff"
    return df 

df=get_reverb_ind(config, df, "test")


display(df.head(10))

,label,idx,compared,1L_multi-stft-mag,1L_stft-mag,1L_multi-wave,1L_wave,1L_logmel,1L_multi-mel,1S_sisdr,...,2S_stoi,3D_pesq,3D_stoi,3D_sisdr,3D_mos_nidiff,3D_pesq_nidiff,3D_stoi_nidiff,3D_sisdr_nidiff,3D_srmr_nidiff,rev_delta
0,anecho+fins,0,prediction:target,0.476317,0.475748,0.053435,0.060730,0.999482,0.159667,1.852891,...,0.890322,0.142396,0.004916,1.880182,0.617181,0.005069,0.022749,1.513630,1.086816,dry2rev
1,anecho+fins,0,prediction:content,0.704501,0.692368,0.084847,0.092072,2.449047,0.258839,-0.228512,...,0.845386,0.940645,0.031773,2.014699,0.613587,0.133918,2.056445,17.869057,1.937228,dry2rev
2,anecho+fins,1,prediction:target,0.803940,0.799786,0.111842,0.148231,1.691724,0.235558,-3.087227,...,0.868637,0.097376,0.029835,8.875996,1.317360,0.048253,0.424020,2.026779,0.785600,rev2dry
3,anecho+fins,1,prediction:content,0.867758,0.870623,0.095291,0.124505,1.539000,0.215473,-5.944984,...,0.858566,0.001621,0.009234,4.540429,0.225493,0.027033,0.387113,1.657860,0.599424,rev2dry
4,anecho+fins,2,prediction:target,0.603527,0.598110,0.053656,0.077396,1.063896,0.159209,-0.237343,...,0.907209,0.141263,0.018162,5.762266,0.002400,0.001132,0.196261,3.545799,0.237573,dry2rev
5,anecho+fins,2,prediction:content,0.667274,0.667963,0.058213,0.075822,3.590947,0.280614,-1.195281,...,0.885286,0.357812,0.015116,5.423813,0.043836,0.050624,1.201276,11.702838,0.587801,dry2rev
6,anecho+fins,3,prediction:target,0.433011,0.423314,0.048414,0.047455,1.621082,0.209412,1.180982,...,0.888140,0.139521,0.005315,0.309434,0.413587,0.006280,0.091485,1.599950,1.860364,smalldiff
7,anecho+fins,3,prediction:content,0.440517,0.440675,0.054298,0.041932,1.667738,0.213590,0.002704,...,0.863087,0.301008,0.008080,1.526373,0.177008,0.008331,0.381816,1.726586,1.451672,smalldiff
8,anecho+fins,4,prediction:target,0.406516,0.404046,0.045226,0.056394,0.954387,0.153667,0.556267,...,0.890774,0.078586,0.026190,3.928321,0.215327,0.022501,0.151066,1.015792,0.752106,rev2dry
9,anecho+fins,4,prediction:content,0.568084,0.560630,0.061615,0.078813,1.639111,0.213655,-2.758394,...,0.842485,0.029152,0.007100,0.057039,0.712749,0.037368,0.749559,1.996424,0.865030,rev2dry


In [22]:
################## GIVE MORE CONCISE TAGS FOR EACH CATEGORY ##################

def impove_categories_tags(df): 
       # add a column to store a shorter tag identifying each category
       df['short_label'] = df['label'].apply(lambda x: x.split('_', 1)[1] if "_" in x else x)
       df['short_label'] = df['short_label'].apply(lambda x: x.replace("checkpoint","ch"))
       df['tag'] = df['short_label']+ ' -> ' + df['compared']
       df['tag'] = df['tag'].apply(lambda x: x.replace("target","tar"))
       df['tag'] = df['tag'].apply(lambda x: x.replace("prediction","pred"))
       # create a custom order of the files so that the plots have similar order as before
       custom_order=["oracle -> tar:anecho", "oracle -> tar:tarclone" , "oracle -> tar:content", "oracle -> tar:style", "anecho+fins -> pred:tar", "dfnet+fins -> pred:tar", "wpe+fins -> pred:tar",
              "c_wunet_stft+wave_0.8_0.2_chbest -> pred:tar", "c_wunet_logmel+wave_0.8_0.2_chbest -> pred:tar", "c_wunet_logmel_1_chbest -> pred:tar", "c_wunet_stft_1_chbest -> pred:tar",
              "c_wunet_stft_1_ch50 -> pred:tar", "c_wunet_stft_1_ch10 -> pred:tar", "c_wunet_stft_1_ch0 -> pred:tar"]
       df['tag'] = pd.Categorical(df['tag'], categories=custom_order, ordered=True)
       df=df.sort_values("compared")
       df=df.drop(columns=['short_label'])
       df=df.sort_values("tag")
       return df

df=impove_categories_tags(df)

In [23]:
# df_sel=df[df["rev_delta"]=="dry2rev"]
df_sel=df[df["rev_delta"]=="rev2dry"]
# df_sel=df
df_g=df_sel.groupby("tag").mean(numeric_only=True).round(decimals=3).reset_index()
cols=df_sel.groupby("tag").groups.keys()
df_g_sel=df_g[['1L_multi-mel', '1L_multi-wave','1L_emb_euc','2L_lsd', '2L_multi-stft', '2L_mcd' , '2S_fwsnr', '3D_pesq', '3D_stoi', '3D_mos_nidiff', '3D_pesq_nidiff', '3D_stoi_nidiff', '3D_sisdr_nidiff', '3D_srmr_nidiff']]
df_g_sel["tag"]=cols
searchfor=["ch0","ch10","ch50","tar:anecho","logmel_1_","stft_1_"]
df_av=df_g_sel[~df_g_sel["tag"].str.contains('|'.join(searchfor))]
df_av.to_csv("results-200325-derev-5k.csv")

df_av

/tmp/ipykernel_570790/114117438.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_g_sel["tag"]=cols


,1L_multi-mel,1L_multi-wave,1L_emb_euc,2L_lsd,2L_multi-stft,2L_mcd,2S_fwsnr,3D_pesq,3D_stoi,3D_mos_nidiff,3D_pesq_nidiff,3D_stoi_nidiff,3D_sisdr_nidiff,3D_srmr_nidiff,tag
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,oracle -> tar:tarclone
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,oracle -> tar:content
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,oracle -> tar:style
4,0.178,0.065,20.261,1.980,1.186,3.920,10.190,0.105,0.017,0.538,0.037,0.273,2.757,1.241,anecho+fins -> pred:tar
5,0.239,0.077,21.307,2.382,1.441,4.638,8.412,0.121,0.061,0.711,0.062,0.342,4.471,1.440,dfnet+fins -> pred:tar
6,0.220,0.078,22.164,2.313,1.384,4.948,8.925,0.195,0.097,0.800,0.104,0.503,6.797,1.554,wpe+fins -> pred:tar
7,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,c_wunet_stft+wave_0.8_0.2_chbest -> pred:tar
8,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,c_wunet_logmel+wave_0.8_0.2_chbest -> pred:tar


In [156]:
import pandas.api.types as ptypes


def compute_delta_similarity(df):
    result_rows = []
    
    for _, row in df.iterrows():
        # Find the row with the same idx but a different tag
        gt_row = df[(df["idx"] == row["idx"]) & (df["tag"] == "oracle -> tar:content")]
        gt_row = gt_row.iloc[0] if not gt_row.empty else None  # Returns None if no match

        # Subtract numerical columns
        new_row = row.copy()
        for col in df.columns:
            if ptypes.is_numeric_dtype(df[col]) or col=="idx" :  # Skip non-numeric columns
                new_row[col] = row[col] - gt_row[col]
        
        result_rows.append(new_row)
    
    return pd.DataFrame(result_rows)

# COMPUTE DELTA_SIMILARITY  
result_rows=compute_delta_similarity(df)

In [28]:
# Group by "tag" and compute mean for all numerical columns

result_rows=result_rows[result_rows["tag"].str.contains("pred:tar")]
av_results = result_rows.groupby("tag").mean(numeric_only=True)


In [31]:
av_results["2L_multi-stft"]

tag
anecho+fins -> pred:tar                           0.043815
c_wunet_logmel+wave_0.8_0.2_chbest -> pred:tar   -0.082657
c_wunet_logmel_1_chbest -> pred:tar              -0.065705
c_wunet_stft+wave_0.8_0.2_chbest -> pred:tar     -0.146017
c_wunet_stft_1_ch0 -> pred:tar                    0.029754
c_wunet_stft_1_ch10 -> pred:tar                  -0.068351
c_wunet_stft_1_ch50 -> pred:tar                  -0.109692
c_wunet_stft_1_chbest -> pred:tar                -0.157420
dfnet+fins -> pred:tar                            0.347753
wpe+fins -> pred:tar                              0.303447
Name: 2L_multi-stft, dtype: float64

In [ ]:
# PLOT AVERAGE METRICS 

# create df with average metric across data points
numeric_df = df.select_dtypes(include='number')
df_g=df.groupby("tag")[numeric_df.columns].mean().reset_index()

# plot average metrics
plt.figure(figsize=(20,60))
N_metrics=len(df_g.columns)
for i, column in enumerate(df_g.columns):
    if column not in ["label", "idx", "compared","short_label","tag"]:    
        N_rows=int(np.ceil(N_metrics/3))
        metriccolor="red" if "D_" in column or "L_" in column else "green"
        plt.subplot(N_rows,3,i+1)
        bars=plt.bar(df_g["tag"],df_g[column], color=metriccolor)
        plt.xticks(rotation=60, ha='right', fontsize="9")
        plt.title("Metric mean: " + column)
        for j,bar in enumerate(bars):
            val4clone=bars[j].get_height()#df_idx[column][df_idx["label"]=="sTarget : sAnecho"][0]
            plt.text(j,val4clone,str("%.2f" % val4clone), horizontalalignment='center',verticalalignment='bottom', fontsize="9")
plt.tight_layout()
plt.show()

